<a href="https://colab.research.google.com/github/CalvHobbes/shop_india/blob/main/llamaindex/price%20comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install beautifulsoup4 requests

In [ ]:
# Import the necessary functions
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance
import json

# Define the product query
query = "iphone16promax"

# Fetch prices from Croma
croma_prices = fetch_prices_from_croma(query)
print("Croma Prices:")
print(json.dumps(croma_prices, indent=4))

# Fetch prices from Reliance Digital
reliance_prices = fetch_prices_from_reliance(query)
print("Reliance Digital Prices:")
print(json.dumps(reliance_prices, indent=4))
